In [1]:

import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sn
import torch


In [2]:
treino = torch.load("data/treino_particle.pt")
teste = torch.load("data/teste_particle.pt")

# Normalizando

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
sc = StandardScaler()
treino_norm = sc.fit_transform(treino[:,0:-1])
y_treino = treino[:,-1]


In [5]:
treino_norm[0]

array([ 8.52577913e+00, -2.13195204e+00,  1.15917927e+00,  4.55274793e+00,
        5.62765844e+00,  6.62813725e+00,  5.63624251e+00,  2.45465204e-01,
       -6.76513533e-02, -3.72452080e-04, -1.36891793e-01, -3.91514539e-02,
        5.93809858e-01, -1.03769838e+00,  3.31891907e-01, -3.66826579e-04])

# Verificando qual otimizador utilizar 

In [6]:
from skorch import NeuralNetBinaryClassifier
from torch import nn
from sklearn.model_selection import GridSearchCV

## Criando modelo

In [7]:
class MLPClassification(nn.Module):
  def __init__(self):
    super().__init__()


    self.layers = nn.Sequential(
        nn.Linear(16,1300),
        nn.ReLU(),
        nn.Linear(1300,700),
        nn.ReLU(),
        nn.Linear(700,350),
        nn.ReLU(),
        nn.Linear(350,200),
        nn.ReLU(),
        nn.Linear(200,50),
        nn.ReLU(),
        nn.Linear(50,1),
        nn.Sigmoid()
    )
  def forward(self,x):
    return self.layers(x)

In [8]:
model = NeuralNetBinaryClassifier(
    MLPClassification(),
    criterion=nn.BCELoss,
    optimizer = torch.optim.Adam,
    verbose=0,
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

In [9]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [10]:
param_grid ={
    'batch_size': [128, 32, 256, 8],
    'max_epochs': [150, 200, 250]
}

In [11]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=None,cv=3)
treino_norm = np.copy(treino_norm).astype(np.float32)
y_treino = np.copy(y_treino).astype(np.float32)
grid_result = grid.fit(treino_norm,y_treino)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.798520 using {'optimizer': <class 'torch.optim.adam.Adam'>}
0.798520 (0.000000) with: {'optimizer': <class 'torch.optim.adam.Adam'>}
0.201481 (0.000001) with: {'optimizer': <class 'torch.optim.sgd.SGD'>}
